In [1]:
import numpy as np
from scipy.io.wavfile import write
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# === 1. Generate training signal ===
sample_rate = 16000
duration = 2  # seconds
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

# Signal = sine wave + noise
signal = np.sin(2 * np.pi * 220 * t) + 0.3 * np.random.randn(*t.shape)
signal = signal.astype(np.float32)

# === 2. Prepare training sequences ===
sequence_length = 100
X, y = [], []

for i in range(len(signal) - sequence_length):
    X.append(signal[i:i + sequence_length])
    y.append(signal[i + sequence_length])

X = np.array(X).reshape((-1, sequence_length, 1))
y = np.array(y)

# === 3. Define and train RNN model ===
model = Sequential([
    SimpleRNN(64, input_shape=(sequence_length, 1), return_sequences=True),
    SimpleRNN(64),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=10, batch_size=32)

# === 4. Generate audio with the trained RNN ===
generated = list(signal[:sequence_length])  # Seed
for _ in range(sample_rate * 2):  # generate 2 seconds
    input_seq = np.array(generated[-sequence_length:]).reshape((1, sequence_length, 1))
    next_sample = model.predict(input_seq, verbose=0)[0][0]
    generated.append(next_sample)

# === 5. Save generated sound to WAV ===
output = np.array(generated)
output = np.clip(output, -1, 1)
write("rnn_generated.wav", sample_rate, (output * 32767).astype(np.int16))

print("Done! File saved as 'rnn_generated.wav'")


Epoch 1/10


C:\Users\Nitro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


997/997 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - loss: 0.1189
Epoch 2/10
997/997 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 0.1011
Epoch 3/10
997/997 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - loss: 0.1010
Epoch 4/10
997/997 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - loss: 0.1008
Epoch 5/10
997/997 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - loss: 0.0981
Epoch 6/10
997/997 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - loss: 0.0984
Epoch 7/10
997/997 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - loss: 0.0987
Epoch 8/10
997/997 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - loss: 0.0966
Epoch 9/10
997/997 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - loss: 0.0986
Epoch 10/10
997/997 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - loss: 0.1057
Done! File saved as 'rnn_generated.wav'
